<a href="https://colab.research.google.com/github/JuniorHZ19/Escalado/blob/main/InterpolacionFramesAudioVideo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown # **1) Instalar Librerias**
# # Clone Real-ESRGAN and enter the Real-ESRGAN
!git clone https://github.com/hzwer/Practical-RIFE.git
%cd Practical-RIFE
! import skvideo.io
!pip3 install -r requirements.txt
!pip3 install --upgrade pip setuptools
!pip install sk-video

In [ ]:
#@markdown # **1) Instalar Modelo**
!mkdir /content/Practical-RIFE/train_log
!gdown --id 1MmaZfFxHVEs0spM4iCB5NDRDM-bWKlIA
!7z e InterpolacionModelo.zip

# Ruta al archivo ZIP que deseas descomprimir
archivo_zip = '/content/Practical-RIFE/InterpolacionModelo.zip'

import zipfile
# Directorio de destino para la descompresión
directorio_destino = '/content/Practical-RIFE/train_log'

with zipfile.ZipFile(archivo_zip, 'r') as zip_ref:
    # Descomprime los archivos en el directorio de destino
    zip_ref.extractall(directorio_destino)


In [ ]:
#@markdown # **1) Montar Driver**
from google.colab import drive, files
drive.mount('/content/drive')

In [ ]:
#@markdown # **3.5) ** Montar el archivo de Google Drive
#@markdown Pon la ruta del archivo en tu google drive "content/drive/MyDrive/... y da click"



drive_mounted = True
drive_path = '/content/drive/MyDrive/aro.mp4'#@param{type: "string"}
file_name = drive_path.split('/')[-1]
input_path = f'/content/drive/MyDrive{drive_path}'
shutil.copy(drive_path,'/content/Practical-RIFE')
print("video path set:\n",input_path)

In [134]:
#@markdown # **3.5 ) ** Convertir a 30fps (Opcional)
#@markdown Si nuestro archvio no esta en 30fps  los videos van a salir mas largos o cortos que no encajaran con el audio

import subprocess

cmd = [
      'ffmpeg',
     '-i',f'/content/Practical-RIFE/{file_name}',

     #'-vf',"scale=3840:2160",

    "-r", '30',  # Establece la tasa de cuadros de salida en 30 fps
    "-c:v", "h264_nvenc",  # Utiliza el códec NVENC H.264
    "-c:a", "aac",  # Códec de audio (puedes ajustarlo según tus necesidades)
    "-strict", "experimental",

      #"-b:v","40M",#birate 5M (720P), 6M (720P:60FPS),8M(1080P), 10M (1080P),15M(1080P:60FPS) ,25M(2K),30M(2K:60FP) 40M(4K:60FP)


       f'/content/Practical-RIFE/30_{file_name}'

]


process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = process.communicate()
if process.returncode != 0:
    print(stderr)

    raise RuntimeError(stderr)
else:

  shutil.move(f'/content/Practical-RIFE/30_{file_name}', f'/content/Practical-RIFE/{file_name}')


In [ ]:
#@markdown # **3.5) ** Interpolar
import shutil
fps=60
muliplicacion=2

#Si es resoluacion 4 poner --scale=0.5
!python3 inference_video.py --exp=1 --fps={fps}  --multi={muliplicacion} --video={file_name} --scale=1

In [ ]:
#@markdown # **3.5) ** Poner audio original

import subprocess

name_file_sin_ext=os.path.splitext(file_name)[0]

fileFullNamme= f'{name_file_sin_ext}_{str(muliplicacion)}X_{fps}fps.mp4'
print(fileFullNamme)


if os.path.exists(f'/content/Practical-RIFE/Audio_{fileFullNamme}'):
    os.remove(f'/content/Practical-RIFE/Audio_{fileFullNamme}')

cmd = [
      'ffmpeg',

     '-i',
     f'/content/Practical-RIFE/{fileFullNamme}',
      '-i',
        f'/content/Practical-RIFE/{file_name}',

        #'-vf',"scale=3840:2160",
       '-c:v',
       'h264_nvenc',
       '-c:a',
        'mp3',
      '-r',
      '60',
       '-b:v','15M', #birate 5M (720P), 6M (720P:60FPS), 10M (1080P),15M(1080P:60FPS) ,25M(2K),30M(2K:60FP) 40M(4K:60FP)
      '-map', '0',
      '-map', '1',

       f'/content/Practical-RIFE/Audio_{fileFullNamme}'

]


process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = process.communicate()
if process.returncode != 0:
    print(stderr)
    raise RuntimeError(stderr)
else:
    print("Se puso el audio ")

In [ ]:
#@markdown # **6)Descargar archivo mejorado**
#@markdown Se pasara el archivo a nuestro drive
import os
from google.colab import files




if drive_mounted:
  if os.path.exists('/content/drive/MyDrive/Real-Rife-Videos') is not True:
    os.makedirs('/content/drive/MyDrive/Real-Rife-Videos')
  shutil.move(f'/content/Practical-RIFE/Audio_{fileFullNamme}',f'/content/drive/MyDrive/Real-Rife-Videos/')
  print(f"MOviendo {fileFullNamme} el archivo a la carpetaReal-Rife-Videos en tu Drive")
